In [1]:
import os
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from google.colab import files
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow

In [2]:
! pip install -q kaggle

In [3]:
files.upload() #upload kaggle.json file

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"minghuaq","key":"663f952bdd8fd276710243a49d89fedd"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
def downloadKaggleData(link):
  name = (link.split(' ')[-1]).split('/')[-1]
  ! {link}
  ! mkdir {name}
  ! unzip {name}.zip -d {name}

In [6]:
downloadKaggleData("kaggle competitions download -c digit-recognizer")

digit-recognizer.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘digit-recognizer’: File exists
Archive:  digit-recognizer.zip
replace digit-recognizer/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
from torch.utils.data import Dataset, DataLoader, Sampler, random_split

class TrainDataset(Dataset):
    def __init__(self, dataLink):
        self.data = pd.read_csv(dataLink).values
    def __len__(self):
        return self.data.shape[0]
    def __getitem__(self, ind):
        x = self.data[ind][1:] / 255.0
        y = self.data[ind][0]
        return x, y
class TestDataset(TrainDataset):
    def __getitem__(self, ind):
        x = self.data[ind] / 255.0
        return x
train_set = TrainDataset("digit-recognizer/train.csv")
train_size = int(0.8 * len(train_set))
val_size = len(train_set) - train_size
train_set, test_set = random_split(train_set, [train_size, val_size])

batch_size = 512
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)

for X, y in train_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# print(train_set[3][1])

number_of_class = 10

Shape of X [N, C, H, W]: torch.Size([512, 784])
Shape of y: torch.Size([512]) torch.int64


In [8]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using {device} device')

class MLP(nn.Module):
  def __init__(self) -> None:
    super(MLP, self).__init__()
    inputlayer = [28*28]
    hiddenlayer = [1024,512,64]
    outputlayer = number_of_class
    hiddenlayer = inputlayer + hiddenlayer

    self.layers = nn.Sequential()
    for i in range(len(hiddenlayer)-1):
      self.layers.add_module(f'linear_{i+1}', nn.Linear(hiddenlayer[i], hiddenlayer[i+1]))
      self.layers.add_module(f'relu_{i+1}', nn.ReLU())
    self.layers.add_module(f'linear_last', nn.Linear(hiddenlayer[-1], outputlayer))

  def forward(self, x):
    x = x.float()
    return self.layers(x)

model = MLP().to(device)
print(model)

Using cpu device
MLP(
  (layers): Sequential(
    (linear_1): Linear(in_features=784, out_features=1024, bias=True)
    (relu_1): ReLU()
    (linear_2): Linear(in_features=1024, out_features=512, bias=True)
    (relu_2): ReLU()
    (linear_3): Linear(in_features=512, out_features=64, bias=True)
    (relu_3): ReLU()
    (linear_last): Linear(in_features=64, out_features=10, bias=True)
  )
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [10]:
def train(dataloader, model, loss_fn, optimizer):
  model.train()

  num_batches = len(dataloader)
  num_items = len(dataloader.dataset)

  total_loss = 0
  total_correct = 0

  losses = []
  for batch_num, (X, y) in enumerate(dataloader):
    optimizer.zero_grad()
    X, y = X.to(device), y.to(device)

    output = model(X)

    loss = loss_fn(output, y)

    total_loss += loss
    total_correct += (torch.argmax(output,dim = 1)== y).sum().item()
    loss.backward()
    losses.append(loss.item())
    optimizer.step()

  train_loss = total_loss/num_batches
  accuracy = total_correct/num_items

  print(f"Average Train loss: {train_loss:7f}, Train accuracy: {accuracy:.2%}")



In [11]:
def test(dataloader, model, loss_fn):
  model.eval()

  num_batches = len(dataloader)
  num_items = len(dataloader.dataset)
  total_loss = 0
  total_correct = 0

  losses = []

  with torch.no_grad():
    for batch_num, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)

      output = model(X)

      loss = loss_fn(output, y)

      total_loss += loss
      total_correct += (torch.argmax(output,dim = 1)== y).sum().item()
      losses.append(loss.item())

  test_loss = total_loss/num_batches
  accuracy = total_correct/num_items

  print(f"Average Test loss: {test_loss:7f}, Test accuracy: {accuracy:.2%}")

In [12]:
epochs = 10
for epoch in range(epochs):
    print(f"Training epoch: {epoch+1}")
    train(train_loader, model, criterion, optimizer)
    test(test_loader, model, criterion)

Training epoch: 1
Average Train loss: 0.629977, Train accuracy: 82.01%
Average Test loss: 0.277845, Test accuracy: 91.60%
Training epoch: 2
Average Train loss: 0.209585, Train accuracy: 93.84%
Average Test loss: 0.167774, Test accuracy: 94.99%
Training epoch: 3
Average Train loss: 0.138132, Train accuracy: 95.92%
Average Test loss: 0.152410, Test accuracy: 95.24%
Training epoch: 4
Average Train loss: 0.098868, Train accuracy: 97.10%
Average Test loss: 0.115223, Test accuracy: 96.67%
Training epoch: 5
Average Train loss: 0.074268, Train accuracy: 97.80%
Average Test loss: 0.106475, Test accuracy: 96.80%
Training epoch: 6
Average Train loss: 0.056402, Train accuracy: 98.33%
Average Test loss: 0.099808, Test accuracy: 97.01%
Training epoch: 7
Average Train loss: 0.039350, Train accuracy: 98.88%
Average Test loss: 0.094680, Test accuracy: 97.17%
Training epoch: 8
Average Train loss: 0.028605, Train accuracy: 99.22%
Average Test loss: 0.094744, Test accuracy: 97.21%
Training epoch: 9
Averag

KeyboardInterrupt: 